In [ ]:
 !pip install pyspark
import pyspark
from pyspark import SparkContext, SparkConf
import csv
import collections
import pandas as pd
import io
import time



# !wget https://tutorialmobile.000webhostapp.com/store_data.csv

def readFile_csv(file):
  dataset = []
  with open(file,'rt')as f:
    data_from_file = csv.reader(f)
    for row in data_from_file:
      dataset.append(row)
  return dataset

def removeReplica(record):
    if(isinstance(record[0], tuple)):
        x1 = record[0]
        x2 = record[1]
    else:
        x1 = [record[0]]
        x2 = record[1]

    if(any(x == x2 for x in x1) == False):
        a = list(x1)
        a.append(x2)
        a.sort()
        result = tuple(a)
        return result 
    else:
        return x1

class Filter():
    def __init__(self):
        self.stages = 1
    def filterForConf(self, item , total):
        if(len(item[0][0]) > len(item[1][0])  ):
            if(self.checkItemSets(item[0][0] , item[1][0]) == False):
                pass
            else:
                return (item)       
        else:
            pass  
        self.stages = self.stages + 1
    def checkItemSets(self, item_1 , item_2):
        if(len(item_1) > len(item_2)):
            return all(any(k == l for k in item_1 ) for l in item_2)
        else:
            return all(any(k == l for k in item_2 ) for l in item_1)

    def calculateConfidence(self, item):
        parent = set(item[0][0])

        if isinstance(item[1][0] , str):
            child  = set([item[1][0]])
        else:
            child  = set(item[1][0])

        parentSupport = item[0][1]
        childSupport = item[1][1]

        support = (parentSupport / childSupport)*100

        return list([ list(child) ,  list(parent.difference(child)) , support ])

    
def scan1(dataset, min_sup):
  conf = SparkConf().setMaster("local").setAppName("word couting")
  sc = SparkContext.getOrCreate(conf=conf)
  data = []
  for i in dataset:
    for j in i:
      data.append(j)
  lblitems = sc.parallelize(dataset)
  wlitems = sc.parallelize(data)
  uniqueItems = wlitems.distinct()

  supportRdd = wlitems.map(lambda item: (item,1))
  supportRdd = supportRdd.reduceByKey(lambda x,y:x+y)
  supportRdd = supportRdd.filter(lambda item: item[1] >= min_sup)
  baseRdd = supportRdd.map(lambda item: ([item[0]] , item[1]))
  supportRdd = supportRdd.map(lambda item: item[0])
  return lblitems, wlitems, uniqueItems, baseRdd, supportRdd

def scan2(lblitems, uniqueItems, baseRdd, supportRdd, min_sup):
  count = 2 
  while supportRdd.isEmpty() == False:
    combined = supportRdd.cartesian(uniqueItems)
    combined = combined.map(lambda item: removeReplica(item))
    combined = combined.filter(lambda item: len(item) == count)
    combined = combined.distinct()
    
    combined_2 = combined.cartesian(lblitems)
    combined_2 = combined_2.filter(lambda item: all(x in item[1] for x in item[0]))
    
    combined_2 = combined_2.map(lambda item: item[0])
    combined_2 = combined_2.map(lambda item: (item , 1))
    combined_2 = combined_2.reduceByKey(lambda x,y:x+y)
    combined_2 = combined_2.filter(lambda item: item[1] >= min_sup)

    baseRdd = baseRdd.union(combined_2)
    
    combined_2 = combined_2.map(lambda item: item[0])
    supportRdd = combined_2
    # print(count ,'. Table has crated... ')
    # print(baseRdd.collect())
    # print(supportRdd.collect())
    count = count+1 
  return baseRdd, count


def filter_with_min_conf(item, maxlen, min_conf):
  if item[len(item) - 1] >= min_conf:
    lentemp = 0
    for i in item:
      if isinstance(i, list):
        lentemp += len(i)
    if lentemp == maxlen:
      return True
    else:
      return False
  else:
    return False

# no pyspack
def filter1_with_sp(dataset, min_sup):
  data = []
  for i in dataset:
    for j in i:
      data.append(j)
  set_of_data = set(data)

  result = []
  for i in list(set_of_data):
    if data.count(i) >= min_sup:
      result.append(( i ,data.count(i)))
  return result

def check_issubset(item, array):
  tmp = []
  for i in array:
    tmp.append(item.issubset(set(i[0])))
  return any(tmp)

def count_set_arr(set_item, dataset):
  count = 0
  for i in dataset:
    if set_item.issubset(set(i)):
      count += 1
  return count

def union_and_count(arr, dataset, min_sup, len_item):
  result = []
  for i in range(0,len(arr)-1):
    for j in range(i+1,len(arr)):
      if len_item == 2:
        temp = set()
        temp.add(arr[i])
        temp.add(arr[j])
      else:
        temp = set(arr[i])
        for k in list(arr[j]):
          temp.add(k)
      count = count_set_arr(temp,dataset)
      if count >= min_sup and not check_issubset(temp, result) and len(temp) == len_item:
        result.append((tuple(temp),count))
  result = set(result)
  result = list(result)
  
  return result

def filter2_with_sp(array, dataset, min_sup):
  # result = []
  result = array
  temp = array
  len_item = 2
  while len(temp) > 1:
    keys = []
    for i in temp:
      keys.append(i[0])

    temp = union_and_count(keys, dataset, min_sup, len_item)
    result += temp
    len_item += 1
  return result, len_item


if __name__ == '__main__':

  # dataset = [['a','c','d'],['b','c','e'],['a','b','c','e'],['b','e']]
  # dataset = [['nhut 1','nhut 3','nhut 4'],['nhut 2','nhut 3','nhut 5'],['nhut 1','nhut 2','nhut 3','nhut 5'],['nhut 2','nhut 5']]

  #---------no pyspark------
  start = time.time()
  dataset = readFile_csv("/content/store_data.csv")
  min_sup = 50
  min_conf = 80

  temp1 = filter1_with_sp(dataset, min_sup)
  # print(temp1)
  # print(len(temp1))
  result, count = filter2_with_sp(temp1, dataset, min_sup)
  print(result)
  print(len(result))
  end = time.time()
  print(end - start)


  #---------use pyspark------
  start = time.time()

  lblitems, wlitems, uniqueItems, baseRdd, supportRdd = scan1(dataset, min_sup)
  baseRdd, count = scan2(lblitems, uniqueItems, baseRdd, supportRdd, min_sup)
  print(baseRdd.collect())
  print(len(baseRdd.collect()))
  # print(len(baseRdd.collect()))
  # maxlen = count - 2
  
  calcuItems = baseRdd.cartesian(baseRdd)
  
  ff = Filter()
  total = calcuItems.count()
  baseRddConfidence = calcuItems.filter(lambda item: ff.filterForConf(item , total))
  # print(baseRddConfidence.collect())
  # baseRddConfidence = baseRddConfidence.map(lambda item: ff.calculateConfidence(item))
  # # print(baseRddConfidence.collect())
  # # baseRddConfidence = baseRddConfidence.filter(lambda item: filter_with_min_conf(item, maxlen, min_conf))
  # # print(baseRddConfidence.collect())

  end = time.time()
  print(end - start)





[('french fries', 1282), ('white wine', 124), ('vegetables mix', 193), ('salt', 69), ('honey', 356), ('milk', 972), ('rice', 141), ('tomato juice', 228), ('pepper', 199), ('chicken', 450), ('frozen smoothie', 475), ('whole weat flour', 70), ('salmon', 319), ('barbecue sauce', 81), ('pancakes', 713), ('shallot', 58), ('antioxydant juice', 67), ('red wine', 211), ('clothes accessories', 63), ('pasta', 118), ('extra dark chocolate', 90), ('oil', 173), ('butter', 226), ('strong cheese', 58), ('magazines', 82), ('eggs', 1348), ('green tea', 991), ('low fat yogurt', 574), ('bug spray', 65), ('light mayo', 204), ('spaghetti', 1306), ('tomatoes', 513), ('brownies', 253), ('carrots', 115), ('parmesan cheese', 149), ('green beans', 65), ('whole wheat pasta', 221), ('shrimp', 536), ('chocolate', 1230), ('herb & pepper', 371), ('strawberries', 160), ('eggplant', 99), ('melons', 90), ('body spray', 86), ('green grapes', 68), ('cereals', 193), ('mint', 131), ('energy drink', 200), ('hot dogs', 243),